# Data Collection ~8/20

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 기본 페이지 URL
base_url = 'https://www.oliveyoung.co.kr/store/main/getBestList.do?t_page=[%ED%94%BC%EC%A7%80%ED%81%B4%EB%A6%AC%EB%84%88]%20%EC%9D%BC%EC%86%8C%20%EC%8A%88%ED%8D%BC%20%EB%A9%9C%ED%8C%85%20%EC%84%B8%EB%B2%94%20%EC%86%8C%ED%94%84%ED%8A%B8%EB%84%88%20150ml%20%EA%B8%B0%ED%9A%8D%20(+%EC%86%9C%2040%EB%A7%A4%20%EC%A6%9D%EC%A0%95)&t_click=GNB&t_gnb_type=%EB%9E%AD%ED%82%B9&t_swiping_type=N'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 현재 날짜
today = datetime.today().strftime('%Y-%m-%d')

# 제품명과 링크를 담을 리스트
data = []

# ul:nth-child(1) ~ ul:nth-child(25)까지 반복
for i in range(1, 26):
    ul_selector = f'#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li'
    products = soup.select(ul_selector)

    for product in products:
        ranking = len(data) + 1
        name = product.select_one('div > div > a > p.tx_name').text.strip()
        link_tag = product.select_one('div > a')
        link = link_tag['href']
        product_url = link

        # 오늘의 특가 여부 확인
        today_special_tag = product.select_one('div > a > span.newOyflag.today')
        today_special = 'Y' if today_special_tag else 'N'

        # 가격 정보 스크랩
        price_org_tag = product.select_one('div > p.prd_price > span.tx_org > span')
        price_now_tag = product.select_one('div > p.prd_price > span.tx_cur > span')

        price_now = price_now_tag.text.strip() if price_now_tag else None
        price_org = price_org_tag.text.strip() if price_org_tag else price_now

        data.append([today, ranking, name, product_url, today_special, price_org, price_now])
        time.sleep(1)

# DataFrame 생성
columns = ['Date', 'Ranking', 'Product', 'Product Url', 'Today Special', 'Price Org', 'Price Now']
df = pd.DataFrame(data, columns=columns)


# 상세 페이지로 들어가 추가 정보 스크랩
for index, row in df.iterrows():
    product_url = row['Product Url']
    product_response = requests.get(product_url, headers=headers)
    product_soup = BeautifulSoup(product_response.text, 'html.parser')

    rating_tag = product_soup.select_one('#repReview > b')
    review_count_tag = product_soup.select_one('#repReview > em')
    category_tag = product_soup.select_one('#dtlCatNm')

    rating = rating_tag.text.strip() if rating_tag else 'N/A'
    review_count = review_count_tag.text.strip() if review_count_tag else 'N/A'
    category = category_tag.text.strip() if category_tag else 'N/A'

    # 추가 정보를 데이터 프레임에 저장
    df.at[index, 'Rating'] = rating
    df.at[index, 'Review Count'] = review_count
    df.at[index, 'Category'] = category

    time.sleep(1)


# 파일 저장
file_name = f'/content/drive/MyDrive/2024Summer Self Study/df_day{datetime.today().strftime("%Y%m%d")}.csv'
df.to_csv(file_name, index=False, encoding='utf-8-sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

Mounted at /content/drive
